<a href="https://colab.research.google.com/github/tejal21190/NLP/blob/main/Webscraping_and_Textual_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project of Webscrapping Texual analysis given by one of the company (Asked to keep their data confidential hence sharing only my work only ) 

In [ ]:
import re
import sys

import requests
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

import string
import pandas as pd
!pip install xlsxwriter
import xlsxwriter
from googleapiclient.discovery import build


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **File Extraction**

In [ ]:

def get_file_id(url):
    file_id = None
    if "/file/d/" in url:
        file_id = url.split("/file/d/")[1].split("/")[0]
    elif "id=" in url:
        file_id = url.split("id=")[1].split("&")[0]
    return file_id

In [ ]:
def get_words(list_urls):
    list_of_words = []
    for url in list_urls:
        # extract the file ID from the URL
        file_id = get_file_id(url)
        # download the file
        download_url = f"https://drive.google.com/uc?id={file_id}"
        response = requests.get(download_url)
        data = response.text
        lines = data.split("\n")
        for line in lines:
            word = line.split("|")[0].replace("\r", "")
            list_of_words.append(word)
    return list_of_words

In [ ]:
def get_url_df(url):
 
    # Read data from a URL and get it as a pandas DataFrame.
    
    file_id = url.split('/')[-2]
    df = pd.read_excel(f"https://drive.google.com/uc?id={file_id}")
    return df

In [ ]:
def extract_article(url):
  response = requests.get(url)

  if response.status_code == 404:
    article=''
  else:
    # Extract the title
    title = BeautifulSoup(response.content, 'html.parser').title.text.strip().split(' | ')[0]  
    # Extract article text
    soup = BeautifulSoup(response.content, 'html.parser')

    # Remove last line of author and college
    pre_data = soup.find_all('pre')
    for pre in pre_data:
        pre.extract()

    text_data = soup.find('div', class_='td-post-content')
    text = text_data.get_text().strip()
    article=title + '\n\n' + text
    return article

# **Textual Analysis**

In [ ]:
def syllable_count(word):
    vowels = 'aeiouy'
    count = 0
    if word.endswith(('ed', 'es')):
        return count
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith('e'):
                count -= 1
    if count == 0:
        count += 1
    return count

In [ ]:
class WebScrap():

    def __init__(self):
        # URLs for all files
        file_url = "https://docs.google.com/spreadsheets/d/" ## Add google drive link of spreadsheet containing links of webpages which need to be webscrapped
        pos_url = [# Add txt file google drive link containing list of positive words]
        neg_url = [# Add txt file google drive link containing list of negative words]
        stop_urls = [# Add txt file google drive link containing list of stop words]
        # Try to fetch data from the URLs
        try:
          self.url_df = get_url_df(file_url)
          self.positive_words = get_words(pos_url)
          self.negative_words = get_words(neg_url)
          self.stop_words = get_words(stop_urls)

        except Exception as e:
          print("Error occurred while fetching data from links")
          raise e
  # Claculate all scores
    def all_score(self, text, url,id):
         # tokenization of text
        tokens = [token for token in word_tokenize(text.lower()) if
                  token not in self.stop_words and token not in string.punctuation]
        # Positive and Negative score
        if tokens:
            pos_score = sum([1 for token in tokens if token in self.positive_words])
            neg_score = sum([1 for token in tokens if token in self.negative_words])

            # Calculate polarity score
            polarity_score = (pos_score - neg_score) / len(tokens)

            # Calculate subjectivity scores
            total_words = len(tokens)
            subjective_score = (pos_score + neg_score) / (total_words + 0.000001)

            # average sentence length
            sentences = sent_tokenize(text)
            number_of_sentences = len(sentences)
            # Word Count
            number_of_words = total_words
            avg_sentence_length = number_of_words / number_of_sentences
            
            # Complex Word Count for more than 2 syllable
            number_of_complex_words = len([word for word in tokens if syllable_count(word) > 2])
            percentage_of_complex_words = (number_of_complex_words / number_of_words) * 100

            # Calculate fog index
            fog_index = 0.4 * (avg_sentence_length + percentage_of_complex_words) if number_of_words > 0 else 0

            # Average Number of Words Per Sentence
            Avg_number_words_per_sentence= number_of_words / number_of_sentences

            # Syllable Count Per Word
            syllable_dict = {}
            for word in tokens:
                syllable_dict[word] = syllable_dict.get(word, 0) + syllable_count(word)
            Syllable_count_per_word=syllable_dict    

            # Personal Pronouns
            pronouns = ['i', 'we', 'my', 'ours', 'us']
            # Regex pattern for not "US"
            pattern = r'\b(?!US\b)[a-zA-Z]+\b'
            tokens = re.findall(pattern, text.lower())
            pronoun_count = sum([1 for token in tokens if token in pronouns])
            
            # Average Word Length
            total_chars = sum(len(word) for word in tokens)
            Avg_word_length = total_chars / total_words

        else:
            pos_score = 0
            neg_score = 0
            polarity_score = 0
            subjective_score = 0
            number_of_sentences = 0
            number_of_words = 0
            number_of_complex_words = 0
            avg_sentence_length = 0
            percentage_of_complex_words = 0
            fog_index = 0
            Avg_number_words_per_sentence=0
            pronoun_count = 0
            Syllable_count_per_word={}
            Avg_word_length=0

        # Return scores as a dictionary
        scores = {
            'URL_ID':id,
            "URL" : url,
            'POSITIVE SCORE': pos_score,
            'NEGATIVE SCORE': neg_score,
            'POLARITY SCORE': polarity_score,
            'SUBJECTIVITY SCORE': subjective_score,
            'AVG SENTENCE LENGTH': avg_sentence_length,
            'PERCENTAGE OF COMPLEX WORDS': percentage_of_complex_words,
            'FOG INDEX': fog_index,
            'AVG NUMBER OF WORDS PER SENTENCE': Avg_number_words_per_sentence,
            'COMPLEX WORD COUNT':number_of_complex_words,
            'WORD COUNT':number_of_words,
            'SYLLABLE PER WORD':Syllable_count_per_word,
            'PERSONAL PRONOUNS': pronoun_count,
            'AVG WORD LENGTH':Avg_word_length
        }
        return scores

    def start_scraping(self):
        results = []
       
        for index, row in self.url_df.iterrows():
            url = (row["URL"])
            id=(row["URL_ID"])
            article_text = extract_article(url)
            # if article_text is not None:
            #     result = self.all_score(text=article_text, url = url,id=id)
            #     result["URL"] = url
            #     results.append(result)
            #     print(result)
            # else:
            if article_text is None:
              article_text=""
            result = self.all_score(text=article_text, url = url,id=id)
            result["URL"] = url
            results.append(result)
            print(result)
            save_text_file(article_text,id)
             
        write_dict_to_xlsx(results,"location"+"Output Data Structure.xlsx") #location where the result must be saved
        

# **Save Output File**

In [ ]:
def write_dict_to_xlsx(data, filename):
    df = pd.DataFrame(data)
    
    # set the hyperlink format
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    url_format = writer.book.add_format({'font_color': 'blue', 'underline': 1})
    
    df.to_excel(writer, index=False)
    
    worksheet = writer.sheets['Sheet1']
    url_col = list(df.columns).index('URL')
    worksheet.set_column(url_col, url_col, 30, url_format)
    writer.save()

# **Saving Article into text file**

In [ ]:
def save_text_file(data,filename):
  file="Location"+str(filename)+".txt" #Location where to save txt file
  a= open(file,'a+')
  a.write(data)
  a.close()

# **Start Scrapping and Textual Analysis**

In [ ]:
w = WebScrap()
w.start_scraping()